In [51]:
import os
from langchain_community.llms import Ollama
from dotenv import load_dotenv
from langchain_community.embeddings import OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import create_retrieval_chain
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain

In [52]:
llm = Ollama(model="llama3.2", base_url="http://127.0.0.1:11434")

In [53]:
embed_model = OllamaEmbeddings(
    model="llama3.2",
    base_url='http://127.0.0.1:11434'
)

In [54]:
text = """

    Mr.beast youtube link =  https://www.youtube.com/channel/UCX6OQ3DkcsbYNE6H8uQQuVA
    """

In [55]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
chunks = text_splitter.split_text(text)

In [56]:
vector_store = Chroma.from_texts(chunks, embed_model)

In [57]:
retriever = vector_store.as_retriever()

In [58]:
chain = create_retrieval_chain(combine_docs_chain=llm,retriever=retriever)

In [59]:
from langchain.prompts import ChatPromptTemplate
# RAG_prompt_template = """

#                 You are an assistant for question-answering tasks. Use the following pieces of 
#                 retrieved context to answer the question. If you don't know the answer, just say 
#                 that you don't know. Use three sentences maximum and keep the answer concise.
#                 and give a youtube link such as.



#                 Context: {context} 
#                 Answer:
#                 """

RAG_prompt_template = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise. If the answer involves a resource, such as a video or article, include a relevant YouTube or website link if available.

### Few-shot Examples ###
Question: How can I learn Python for beginners?
Context: A YouTube tutorial on Python basics and an online course are highly recommended.
Answer: You can start with this YouTube tutorial: https://www.youtube.com/watch?v=rfscVS0vtbw. Alternatively, try an online course like Codecademy’s Python program.

Question: What is quantum computing?
Context: A website explaining quantum mechanics and a YouTube video providing an introduction.
Answer: Quantum computing leverages the principles of quantum mechanics to process information. Check out this introduction video: https://www.youtube.com/watch?v=OWJCfOvochA or read more here: https://quantum.country/.

### Your Turn ###
Context: {context}
Answer:
"""

# Question: {question} 

# RAG Prompt template
retrieval_qa_chat_prompt = ChatPromptTemplate(
    ("human",RAG_prompt_template),
)

# retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [60]:
combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)

In [61]:
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)
retrieval_qa_chat_prompt

ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='human'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="\nYou are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise. If the answer involves a resource, such as a video or article, include a relevant YouTube or website link if available.\n\n### Few-shot Examples ###\nQuestion: How can I learn Python for beginners?\nContext: A YouTube tutorial on Python basics and an online course are highly recommended.\nAnswer: You can start with this YouTube tutorial: https://www.youtube.com/watch?v=rfscVS0vtbw. Alte

In [64]:
response = retrieval_chain.invoke({"input":"Coco and Mango"})
response

{'input': 'Coco and Mango',
 'context': [Document(metadata={}, page_content='Mr.beast youtube link is https://www.youtube.com/channel/UCX6OQ3DkcsbYNE6H8uQQuVA'),
  Document(metadata={}, page_content='Mr.beast youtube link is https://www.youtube.com/channel/UCX6OQ3DkcsbYNE6H8uQQuVA'),
  Document(metadata={}, page_content='Mr.beast youtube link =  https://www.youtube.com/channel/UCX6OQ3DkcsbYNE6H8uQQuVA'),
  Document(metadata={}, page_content='Mr.beast youtube link =  https://www.youtube.com/channel/UCX6OQ3DkcsbYNE6H8uQQuVA')],
 'answer': "I think there's an error in the provided context. The correct format for answering a question would be to respond with three sentences maximum, like this:\n\nYou: Mr.beast youtube link is https://www.youtube.com/channel/UCX6OQ3DkcsbYNE6H8uQQuVA\nAssistant: MrBeast's YouTube channel is known for its generosity and unique content. His videos often feature giveaways, challenges, and other exciting activities. You can find his channel by visiting the provi